In [4]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate
import pandas as pd

In [ ]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [1]:
!./setuprds.sh
!./setupdynamo.sh

Setting up users
CREATE ROLE
CREATE ROLE
CREATE ROLE
Setting up account ledger in RDS local
CREATE SCHEMA
CREATE TABLE
CREATE INDEX
CREATE INDEX
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
GRANT
Setting up transaction ledger in RDS local
CREATE SCHEMA
CREATE TABLE
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
Setting up float ledger in RDS local
CREATE SCHEMA
CREATE TABLE
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
GRANT
Creating the DynamoDB table that holds the client floats
{
    "StackId": "arn:aws:cloudformation:us-east-1:123456789:stack/client-float-table/9cbbf9fe-b2a4-429a-95af-efce442d4a43"
}
Adding in item for core, ZAR wholesale float


In [ ]:
!./deploylambdas.sh

In [5]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [6]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [ ]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [ ]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [ ]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [ ]:
def generate_saving_transaction(account_id, ref_amount = 100, float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id
    }
    return test_saving_dict

In [ ]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='add-savings', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Tx for account %s persisted with new balance %s' % (tx['accountId'], persisted_tx['currentBalance']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [ ]:
print_deployed_functions()

In [ ]:
accounts = create_number_accounts(number_accounts = 1)

In [ ]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts[:100]]
print(tabulate.tabulate(account_info, headers = 'keys'))

In [ ]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='add-savings', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

In [ ]:
account_balances = seed_savings_for_accounts(accounts[:100], tx_per_account = 3, base_amount = 100 * 100 * 100)

In [ ]:
def merge_dataframes(accounts, account_balances):
    df = pd.DataFrame(account_balances)
    df = df[df['currentBalance'] == df.groupby('accountId')['currentBalance'].transform('max')]
    df = df[['accountId', 'currentBalance']]
    df['currentBalance'] = pd.to_numeric(df['currentBalance'])
    df['balanceRand'] = df['currentBalance']/10000
    account_name_df = pd.DataFrame(accounts)[['accountId', 'floatId', 'userFamilyName', 'userFirstName']]
    account_name_df['Holder Name'] = account_name_df['userFirstName'] + ' ' + account_name_df['userFamilyName']
    account_name_df.set_index('accountId')
    df.set_index('accountId')
    merged_df = pd.merge(account_name_df, df)[['accountId', 'floatId', 'Holder Name', 'balanceRand']]
    return merged_df

In [ ]:
merged_df = merge_dataframes(accounts, account_balances)

In [ ]:
merged_df

In [7]:
accrualDict = {
    'clientId': 'zar_client_co',
    'floatId': 'zar_cash_float',
    'accrualAmount': 100 * 100 * 100,
    'currency': 'ZAR',
    'unit': 'HUNDREDTH_CENT',
    'backingEntityIdentifier': 'tx-id-backing'
}
local_lambda.invoke(FunctionName='float-api-local-accrue', InvocationType='RequestResponse', 
                                           Payload=json.dumps(accrualDict, default=str))

ClientError: An error occurred (InternalFailure) when calling the Invoke operation (reached max retries: 4): Error executing Lambda function arn:aws:lambda:eu-west-1:000000000000:function:float-api-local-accrue: Lambda process returned error status code: 1. Output:
[32mSTART RequestId: 876cddd5-82f4-10c9-681d-70baf392976a Version: $LATEST[0m
2019-05-31T15:26:20.235Z pluto:dynamo:main Set endpoint for DynamoDB:  http://172.17.0.1:4569
2019-05-31T15:26:20.278Z pluto:float:handler Initiated execution, attempting to reach DynamoDB ...
2019-05-31T15:26:20.280Z pluto:dynamo:main Transformed key:  { client_id: 'zar_client_co', float_id: 'zar_cash_float' }
2019-05-31T15:26:20.281Z pluto:dynamo:main Passing parameters to docClient:  { TableName: 'ClientFloatTable',
  Key: { client_id: 'zar_client_co', float_id: 'zar_cash_float' },
  ProjectionExpression: 'bonus_pool_share_of_accrual, bonus_pool_system_wide_id, client_share_of_accrual, client_share_of_system_wide_id' }
2019-05-31T15:26:20.281Z